In [ ]:
import time
import jax
import jax.numpy as jnp
from jax import grad, vmap
from tqdm.notebook import tqdm
from netket.experimental.driver.vmc_srt import VMC_SRt



# 假设你的模型对象为 model，且其 .apply 方法接收 {"params": params} 以及输入样本
# 下面计算单个样本下的log(ψ)梯度，返回展平后的梯度向量（形状为 (Np,)）
def compute_log_derivative(params, sample):
    # 这里定义 log_psi，为避免log(0)加一个小常数
    def log_psi(p, x):
        psi = model.apply({"params": p}, x)  # 模型输出复数
        return jnp.log(jnp.abs(psi) + 1e-12)
    flat_grad, _ = jax.flatten_util.ravel_pytree(grad(log_psi)(params, sample))
    return flat_grad

# 批量计算log-derivatives，返回矩阵 O 形状 (N_samples, Np)
def compute_log_derivative_batch(params, samples):
    return vmap(lambda x: compute_log_derivative(params, x))(samples)

# 定义MinSR驱动（继承自 VMC_SRt），实现公式 δθ = O† · (T⁻¹ · ϵ)，T = O_centered · O_centered†
class MinSR_VMC(VMC_SRt):
    def __init__(self, reference_energy, delta_tau, **kwargs):
        """
        delta_tau: imaginary-time步长，用于构造ϵ = -delta_tau*(Eloc-<E>)/sqrt(N_samples)
        其他参数传递给父类VMC_SRt
        """
        super().__init__(**kwargs)
        self.delta_tau = delta_tau
        self.reference_energy = reference_energy

    def _step_with_state(self, state):
        # state.parameters: 当前模型参数（PyTree格式）
        # state.samples: 一批 Monte Carlo 采样，形状 (N_samples, ...)
        # 假设 state.energy 提供局部能量 state.energy.local (形状 (N_samples,))
        # 以及能量均值 state.energy.mean
        
        N_samples = state.samples.shape[0]
        # 计算 log-derivatives 矩阵 O，形状 (N_samples, Np)
        O = compute_log_derivative_batch(state.parameters, state.samples)
        # 中心化：每列减去均值
        O_mean = jnp.mean(O, axis=0, keepdims=True)
        O_centered = O - O_mean

        # 构造 ϵ = -delta_tau*(Eloc - <E>)/sqrt(N_samples)
        eps = - self.delta_tau * (state.energy.local - state.energy.mean) / jnp.sqrt(N_samples)

        # 构造 T = O_centered · O_centered†，形状 (N_samples, N_samples)
        T = O_centered @ jnp.conjugate(O_centered.T)
        # 计算 T 的伪逆（可根据需要调整 rcond 参数，此处设为 1e-12 ）
        T_inv = jnp.linalg.pinv(T, rcond=1e-12)
        
        # 计算更新向量 δθ = O_centered† · (T⁻¹ · eps)
        delta_theta = jnp.conjugate(O_centered.T) @ (T_inv @ eps)  # 形状 (Np,)

        # 更新参数：此处使用预先设置的优化器（如 Sgd）
        new_params = self.optimizer.update(delta_theta, state.parameters)
        new_state = state.replace(parameters=new_params)
        return new_state

    def run(self, n_iter, out=None):
        """
        运行优化，并使用 tqdm 进度条显示当前进度，
        显示内容包括迭代次数、当前能量均值和能量误差等信息。
        """
        pbar = tqdm(total=n_iter, desc="MinSR Optimization", leave=True)
        for i in range(n_iter):
            self.advance(1)
            energy_mean = self.energy.mean
            energy_var = self.energy.variance
            energy_err  = self.energy.error_of_mean
            relative_error = abs((energy_mean - self.reference_energy) / self.reference_energy) * 100
            pbar.set_postfix({
                'Energy': f'{energy_mean:.6f}',
                'E_err': f'{energy_err:.6f}',
                'E_var': f'{energy_var:.6f}',
                'Rel_err(%)': f'{relative_error:.4f}',
            })
            pbar.update(1)
        pbar.close()
        return self

# ----- 以下为示例调用 -----
# 假设已有以下变量：
# ha: 已定义的哈密顿量（JAX operator 格式）
# optimizer: 已定义的优化器（例如 nk.optimizer.Sgd(learning_rate=0.05)）
# vqs: 通过 nk.vqs.MCState 构造的变分量子态
# model: 你的 neural network 模型（MinSR驱动内部需要调用 model.apply）
reference_energy = -16.2618
delta_tau = 1e-3  # 设定 imaginary-time 步长，根据需要调整
optimizer = nk.optimizer.Sgd(learning_rate=0.05)
minsr_driver = MinSR_VMC(
    delta_tau=delta_tau,
    reference_energy=reference_energy,
    hamiltonian=ha,
    optimizer=optimizer,
    diag_shift=0.01,
    variational_state=vqs
)

start_time = time.time()
minsr_driver.run(n_iter=1000)
end_time = time.time()
print(f"MinSR Optimization time: {end_time - start_time:.2f} seconds")
